<a href="https://colab.research.google.com/github/samsoe/mpg_notebooks/blob/master/YVP_Ground_Cover_Data_Wrangle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

*R Notebook*

# README

* Readme fixed plot vegetation data - [Ground Cover Data](https://docs.google.com/document/d/16-Aq8u9Rudd78fSzfjvpCXyQgE-BstC-d2PjYfmLtcw/edit#heading=h.a2v9q2quarai)

# Load Tools

In [1]:
library(tidyverse)

── Attaching packages ─────────────────────────────────────── tidyverse 1.3.0 ──

✔ ggplot2 3.3.0     ✔ purrr   0.3.3
✔ tibble  3.0.0     ✔ dplyr   0.8.5
✔ tidyr   1.0.2     ✔ stringr 1.4.0
✔ readr   1.3.1     ✔ forcats 0.5.0

── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()



# Source

In [0]:
# '2020-04-28_yvp_ground_cover.csv'
src = 'https://drive.google.com/uc?id=1JXlFaVP1UxuKmnHju8r6T-8ssrPYO9dV'

In [0]:
df <- read.csv(file = src)

In [4]:
head(df, n=2)

,plot_code,date,subplot,groundcover_type,groundcover_pct
,<fct>,<fct>,<int>,<fct>,<int>
1,YVP 10,2017-06-09,1,BG,1
2,YVP 10,2017-06-09,1,BV,80


# Wrangle

## Structure Columns

### plot_code

In [0]:
# convert to string
df$plot_code <- as.character(df$plot_code)

### plot_loc

In [0]:
# detect "N" in 'plot_code' and write to new column 'plot_loc'
df <- df %>%
  mutate(plot_loc = ifelse(str_detect(plot_code, "N"), "N", NA))

In [0]:
# # strip "N" from 'plot_code' if present
# df$plot_code <- str_remove(df$plot_code, "N")

In [0]:
# reorder columns
df <- df[, c(1,6,2,3,4,5)]

### plot_rep

In [0]:
# detect "A", "B", "C" characters in plot_code and if present write to 'plot_rep'
df <- df %>%
  mutate(plot_rep = case_when(str_detect(plot_code, "A")~"A",
                              str_detect(plot_code, "B")~"B",
                              str_detect(plot_code, "C")~"C"))

In [0]:
# # strip "A", "B", "C" from plot_code
# df$plot_code <- str_remove(df$plot_code, "[ABC]")

In [0]:
# reorder columns
df <- df[,c(1,2,7,3,4,5,6)]

### plot_num

In [0]:
# use digital values from 'plot_code' and to populate 'plot_num'
df <- df %>%
  mutate(plot_num = str_extract(plot_code, "[:digit:].*"))

In [0]:
# reorder columns
df <- df[,c(1,2,3,8,4,5,6,7)]

### date

In [0]:
# convert to date
df$date <- as.Date(df$date)

### subplot

In [0]:
typeof(df$subplot)

[1] "integer"

### groundcover_type

In [0]:
# convert to string
df$groundcover_type <- as.character(df$groundcover_type)

### groundcover_pct

In [0]:
typeof(df$groundcover_pct)

[1] "integer"

## Review 'groundcover_type' levels

Make sure that all levels of `groundcover_type` are present in each subplot of the data sheet (this is important for data collection)

In [0]:
head(df)

,plot_code,plot_loc,plot_rep,plot_num,date,subplot,groundcover_type,groundcover_pct
,<chr>,<chr>,<chr>,<chr>,<date>,<int>,<chr>,<int>
1,YVP 10,NA,NA,10,2017-06-09,1,BG,1
2,YVP 10,NA,NA,10,2017-06-09,1,BV,80
3,YVP 10,NA,NA,10,2017-06-09,1,L,10
4,YVP 10,NA,NA,10,2017-06-09,1,WD,0
5,YVP 10,NA,NA,10,2017-06-09,1,M,5
6,YVP 10,NA,NA,10,2017-06-09,1,LIC,4


In [0]:
# display distinct 'groundcover_type' for reference
df %>%
  distinct(groundcover_type) %>%
  arrange(groundcover_type)

groundcover_type
<chr>
BG
BV
G
L
LIC
M
R
WD


In [0]:
groundcover_counts <- df %>%
  group_by(year = as.numeric(substring(date,0,4)), plot_code, subplot) %>%
  arrange(year, plot_code, subplot, groundcover_type) %>%
  summarize(groundcover_type_counted = n())

In [0]:
head(groundcover_counts)

year,plot_code,subplot,groundcover_type_counted
<dbl>,<chr>,<int>,<int>
2017,YVP 10,1,8
2017,YVP 10,2,8
2017,YVP 10,3,8
2017,YVP 10,4,8
2017,YVP 10,5,8
2017,YVP 10,6,8


In [0]:
unique(groundcover_counts$groundcover_type_counted)

[1] 8 7 9

In [0]:
groundcover_counts %>%
  filter(groundcover_type_counted != 8) %>%
  arrange(groundcover_type_counted, year)

year,plot_code,subplot,groundcover_type_counted
<dbl>,<chr>,<int>,<int>
2017,YVP 355,1,7
2017,YVP 481,1,7
2018,YVP N278,5,7
2018,YVP N57,8,7
2019,YVP 481,1,7
2018,YVP N278,6,9
2018,YVP N57,2,9


In [0]:
# example of short one 'ground_cover'
df %>%
  filter(as.numeric(substring(date,0,4)) == 2017,
          plot_code == 'YVP 355',
          subplot == 1)

# for all groundcover_type_counted find missing category and create row with groundcover_pct = NA
# BG in this case

plot_code,plot_loc,plot_rep,plot_num,date,subplot,groundcover_type,groundcover_pct
<chr>,<chr>,<chr>,<chr>,<date>,<int>,<chr>,<int>
YVP 355,NA,NA,355,2017-06-02,1,BV,75
YVP 355,NA,NA,355,2017-06-02,1,L,10
YVP 355,NA,NA,355,2017-06-02,1,WD,0
YVP 355,NA,NA,355,2017-06-02,1,M,5
YVP 355,NA,NA,355,2017-06-02,1,LIC,10
YVP 355,NA,NA,355,2017-06-02,1,G,1
YVP 355,NA,NA,355,2017-06-02,1,R,0


In [0]:
# 2018	YVP N278	6
# example of one extra 'groundcover_type'
df %>%
  filter(as.numeric(substring(date,0,4)) == 2018,
          plot_code == 'YVP N278',
          subplot == 6) %>%
  arrange(groundcover_type)

# remove row YVP N278	N	NA	278	2018-05-28	6	M	15

plot_code,plot_loc,plot_rep,plot_num,date,subplot,groundcover_type,groundcover_pct
<chr>,<chr>,<chr>,<chr>,<date>,<int>,<chr>,<int>
YVP N278,N,NA,278,2018-05-28,6,BG,1
YVP N278,N,NA,278,2018-05-28,6,BV,20
YVP N278,N,NA,278,2018-05-28,6,G,4
YVP N278,N,NA,278,2018-05-28,6,L,25
YVP N278,N,NA,278,2018-05-28,6,LIC,10
YVP N278,N,NA,278,2018-05-28,6,M,15
YVP N278,N,NA,278,2018-05-28,6,M,40
YVP N278,N,NA,278,2018-05-28,6,R,0
YVP N278,N,NA,278,2018-05-28,6,WD,0


In [0]:
# 2018	YVP N57	2
df %>%
  filter(as.numeric(substring(date,0,4)) == 2018,
          plot_code == 'YVP N57',
          subplot == 2) %>%
  arrange(groundcover_type)

# remove row YVP N57	N	NA	57	2018-06-08	2	M	5

plot_code,plot_loc,plot_rep,plot_num,date,subplot,groundcover_type,groundcover_pct
<chr>,<chr>,<chr>,<chr>,<date>,<int>,<chr>,<int>
YVP N57,N,NA,57,2018-06-08,2,BG,3
YVP N57,N,NA,57,2018-06-08,2,BV,10
YVP N57,N,NA,57,2018-06-08,2,G,0
YVP N57,N,NA,57,2018-06-08,2,L,40
YVP N57,N,NA,57,2018-06-08,2,LIC,3
YVP N57,N,NA,57,2018-06-08,2,M,35
YVP N57,N,NA,57,2018-06-08,2,M,5
YVP N57,N,NA,57,2018-06-08,2,R,0
YVP N57,N,NA,57,2018-06-08,2,WD,10
